# Calculating Service rate of (s) servers from LOS - Hypoexponential/Erlang distribution

After reading few papers, I have come to believe that LOS data can be best represented by a Phase-type distribution. This can be a mixed distribution of hypo and hyper-exponential distributions. However, considering a simple case of hypoexponential distribution (also called Erlang distribution), adn given the number of ED visits for arrival rate calculation, we can find the service rate of a single server as ($\mu$) and ($s\mu$) for s servers.

Hypoexponential distribution can have s servers with different exponential service rates. However, scipy doesn't have function for hypoexponential distribution. So we assume an Erlang distribution here. Only assumption as a result of using Erlang distribution would be similar exponential service rates of s servers.

Reference: 

[1] M. Faddy, N. Graves, and A. Pettitt, “Modeling Length of Stay in Hospital and Other Right Skewed Data: Comparison of Phase-Type, Gamma and Log-Normal Distributions,” Value in Health, vol. 12, no. 2, pp. 309–314, Mar. 2009, doi: https://doi.org/10.1111/j.1524-4733.2008.00421.x.

In [42]:
import pandas as pd
import numpy as np
from scipy.stats import kstest, lognorm
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
%matplotlib inline

Importing data as dataframe

In [2]:
xls = pd.ExcelFile("data/provisional-emergency-department-visits-apr-sep-2022-provisional-data-tables-en.xlsx")
xls.sheet_names

['ED Visits, 2022–2023',
 'Notes to readers',
 'Table of contents',
 '1 ED visits, LOS',
 '2 ED visits, month, age and sex',
 '3 Top 10 main problems']

In [3]:
df = pd.read_excel("data/provisional-emergency-department-visits-apr-sep-2022-provisional-data-tables-en.xlsx", xls.sheet_names[3])
df.head(5)

,"Screen reader users: There is 1 table on this tab called Table 1: Number of ED visits and length of stay by Canadian Triage and Acuity Scale (CTAS) levels and admitted cases, participating provinces/territories, NACRS, April 2022 to September 2022. It begins at cell A5 and ends at cell K15. The notes begin in cell A16, coverage information begins in cell A24 and the source begins in cell A26. A link back to the table of contents is in cell A2.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Back to the Table of contents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Table 1 Number of ED visits and length of sta...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,ED volumes,NaN,NaN,NaN,"Median \n(50% spent less, in hours)",NaN,NaN,"90th percentile \n(90% spent less, in hours)",NaN,NaN
3,Province/territory*,\nTotal\nED volumes,CTAS levels I–III (discharged)†\nED volumes,CTAS levels IV–V (discharged)‡\nED volumes,\nAdmitted§\nED volumes,CTAS levels I–III (discharged)†\nMedian (50% s...,CTAS levels IV–V (discharged)‡\nMedian (50% sp...,"\nAdmitted§\nMedian (50% spent less, in hours)",CTAS levels I–III (discharged)†\n90th percenti...,CTAS levels IV–V (discharged)‡\n90th percentil...,"\nAdmitted§\n90th percentile (90% spent less, ..."
4,P.E.I.**,34322,13647,8839,3590,4.7,5,19.5,10.3,10.8,81.2


In [4]:
df = df.rename(columns = {"Screen reader users: There is 1 table on this tab called Table 1: Number of ED visits and length of stay by Canadian Triage and Acuity Scale (CTAS) levels and admitted cases, participating provinces/territories, NACRS, April 2022 to September 2022. It begins at cell A5 and ends at cell K15. The notes begin in cell A16, coverage information begins in cell A24 and the source begins in cell A26. A link back to the table of contents is in cell A2.": "Province/Territory",
                         "Unnamed: 1": "Total ED volume",
                         "Unnamed: 2": "ED volume CTAS I-III",
                         "Unnamed: 3": "ED volume CTAS IV-V",
                         "Unnamed: 4": "ED volume Admitted",
                         "Unnamed: 5": "50th percentile LOS - CTAS I-III",
                         "Unnamed: 6": "50th percentile LOS - CTAS IV-V",
                         "Unnamed: 7": "50th percentile LOS - Admitted",
                         "Unnamed: 8": "90th percentile LOS - CTAS I-III",
                         "Unnamed: 9": "90th percentile LOS - CTAS IV-V",
                         "Unnamed: 10": "90th percentile LOS - Admitted"})

In [5]:
df = df.fillna('False')
df.head(5)

,Province/Territory,Total ED volume,ED volume CTAS I-III,ED volume CTAS IV-V,ED volume Admitted,50th percentile LOS - CTAS I-III,50th percentile LOS - CTAS IV-V,50th percentile LOS - Admitted,90th percentile LOS - CTAS I-III,90th percentile LOS - CTAS IV-V,90th percentile LOS - Admitted
0,Back to the Table of contents,False,False,False,False,False,False,False,False,False,False
1,Table 1 Number of ED visits and length of sta...,False,False,False,False,False,False,False,False,False,False
2,False,ED volumes,False,False,False,"Median \n(50% spent less, in hours)",False,False,"90th percentile \n(90% spent less, in hours)",False,False
3,Province/territory*,\nTotal\nED volumes,CTAS levels I–III (discharged)†\nED volumes,CTAS levels IV–V (discharged)‡\nED volumes,\nAdmitted§\nED volumes,CTAS levels I–III (discharged)†\nMedian (50% s...,CTAS levels IV–V (discharged)‡\nMedian (50% sp...,"\nAdmitted§\nMedian (50% spent less, in hours)",CTAS levels I–III (discharged)†\n90th percenti...,CTAS levels IV–V (discharged)‡\n90th percentil...,"\nAdmitted§\n90th percentile (90% spent less, ..."
4,P.E.I.**,34322,13647,8839,3590,4.7,5,19.5,10.3,10.8,81.2


In [40]:
bc = df[df["Province/Territory"].str.match(r"B.C.(.*?)")]

# Claculations for CTAS I - III only.
mean_50 = float(bc["50th percentile LOS - CTAS I-III"])
mean_90 = float(bc["90th percentile LOS - CTAS I-III"])
ed_visits = int(bc["ED volume CTAS I-III"])

print(mean_50, mean_90, ed_visits)
bc

3.9 8.0 430213


,Province/Territory,Total ED volume,ED volume CTAS I-III,ED volume CTAS IV-V,ED volume Admitted,50th percentile LOS - CTAS I-III,50th percentile LOS - CTAS IV-V,50th percentile LOS - Admitted,90th percentile LOS - CTAS I-III,90th percentile LOS - CTAS IV-V,90th percentile LOS - Admitted
11,B.C.**,870924,430213,266125,105106,3.9,2.7,17.8,8,5.6,52.8


## (constant) Arrival Rate calculation

Now, we can calculate a constant arrival rate by 2 techniques:

1. By proving that LOS is a log-normal distribution, given the 50th and 90th percentiles. However, in this case, the arrival of second patient is after the treatment completion (here, time of exit from ED) of the first patient.

2. We use the number of ED arrivals (per 6 month time period) data and divide it by the period of time to find a consatnt arrival rate per minute or per hour.

The 1st method is discussed and implemented in the constant-arrival-rate-calculation.ipynb notebook. This notebook uses the 2nd method to calculate arrival rate.

In [39]:
# Arrival rate (per minute) from number of ED visits - CTAS I-III
time_period = 6*30*24*60
arrival_rate = ed_visits/time_period
print(arrival_rate)

1.65977237654321


## Maximum Likelihood Estimation to fit LOS as a hypoexponential/Erlang distribution

In [ ]:
n_samples = ed_visits
samples = erlang(data)

In [51]:
# Define the percentile equation as a function
def percentile_eqn(x, *args):
    lambd, p, k = args
    return 1 - np.exp(-(p/100)*lambd*k*x) - p/100

# Define the objective function to minimize
def obj_fn(x, *args):
    return percentile_eqn(x, *args)**2

try:
    # Solve for the shape and scale parameters using the percentiles and arrival rate
    sol = fsolve(obj_fn, x0=[5, 5], args=(arrival_rate, mean_50, 2))
    k = round(sol[0])
    theta = round(sol[1], 2)
    
    if k < 1 or theta <= 0:
        print("Invalid Erlang distribution")
    else:
        expected_mean_50 = erlang.ppf(0.5, k, scale=theta)
        expected_mean_90 = erlang.ppf(0.9, k, scale=theta)

        print("Shape parameter (k) = ", k)
        print("Scale parameter (theta) = ", theta)
        print(expected_mean_50, expected_mean_90)
except:
    print("Error: Could not estimate parameters")

Invalid Erlang distribution
